In [1]:
import torch
import re
import unicodedata
import numpy as np

In [2]:
data_dir = './data/train_logs_split_20.txt'
with open(data_dir, 'r') as f:
    data = f.read().splitlines()

In [11]:
ano_dir = './data/test_logs_split_20.txt'
with open(ano_dir, 'r') as f:
    test_data = f.read().splitlines()

In [12]:
len(test_data)

2463201

In [9]:
len(data)

8712428

In [10]:
len(data)*0.1

871242.8

In [16]:
with open('./data/test/train_logs_split_20_test_size', 'w') as f:
    for log in data[:len(test_data)]:
        f.write(log + '\n')

In [18]:
with open('./data/test/val_logs_split_20_test_size', 'w') as f:
    for log in data[len(test_data):len(test_data)+round(len(test_data)*0.1)]:
        f.write(log + '\n')

In [513]:
len(data)

8712428

In [460]:
data_dir = './data/test_logs_split_20.txt'
with open(data_dir, 'r') as f:
    test_data = f.read().splitlines()

In [511]:
# data split

groups = [[k, list(list(zip(*list(g)))[0])] for k, g in 
          groupby(zip(data, blk), lambda x: x[1])]

In [548]:
val_groups = groups[-round(len(groups)*0.1):]
val_groups = [x for y in val_groups for x in y[1]]
len(val_groups)

888182

In [551]:
val_blk = [get_blk_id(log) for log in val_groups]

In [552]:
len(val_blk)

888182

In [549]:
train_groups = groups[:-round(len(groups)*0.1)]
train_groups = [x for y in train_groups for x in y[1]]
len(train_groups)

7824246

In [553]:
train_blk = [get_blk_id(log) for log in val_groups]

In [554]:
with open('./data/train_logs_split_20_10', 'w', encoding='utf-8') as f:
    for log in train_groups:
        f.write(log + '\n')

In [568]:
with open('./data/train_logs_split_20_10_tmp', 'w', encoding='utf-8') as f:
    for log in train_groups[:10000000]:
        f.write(log + '\n')

In [555]:
with open('./data/val_logs_split_20_10', 'w', encoding='utf-8') as f:
    for log in val_groups:
        f.write(log + '\n')

In [569]:
with open('./data/val_logs_split_20_10_tmp', 'w', encoding='utf-8') as f:
    for log in val_groups[:1000000]:
        f.write(log + '\n')

In [562]:
train_groups[10000]

'081109 203727 223 INFO dfs.DataNode$DataXceiver: Receiving block blk_-6343486872402960301 src: /10.251.67.211:37607 dest: /10.251.67.211:50010'

In [563]:
data[10000]

'081109 203727 223 INFO dfs.DataNode$DataXceiver: Receiving block blk_-6343486872402960301 src: /10.251.67.211:37607 dest: /10.251.67.211:50010'

In [ ]:
train_groups

In [499]:
len(data[:100])

100

In [31]:
def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )


def normalize_string(s):
    #s = unicode_to_ascii(s.lower().strip())
    s = re.sub(r"([.!?])", r" ", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def log_parser(log):
    log = unicode_to_ascii(log.lower().strip())
    
    id_regex = re.compile('blk_.\d+')
    ip_regex = re.compile('\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}(:\d{1,5})?')
    num_regex = re.compile('\d*\d')

    tmp = re.sub(id_regex, "BLK", log)
    tmp = re.sub(ip_regex, "IP", tmp)
    tmp = re.sub(num_regex, "NUM", tmp)
    
    return normalize_string(tmp)

def get_blk_id(log):
    id_extractor = re.compile('blk_.\d+')
    return id_extractor.search(log).group()

In [311]:
blk = [get_blk_id(log) for log in data]

In [ ]:
list(zip(*groups[0]))[0]

In [475]:
groups = {k:[log_parser(x[0]) for x in list(g)] for k, g in groupby(zip(data, blk), lambda x: x[1])}

KeyboardInterrupt: 

In [88]:
groups[list(groups.keys())[112]]

['NUM NUM NUM info dfs datanode dataxceiver receiving block BLK src IP dest IP']

In [104]:
groups = [[l for l in list(g) key[0]:log for log, key in zip(*l)] for k, g in groupby(zip(data, blk), lambda x: x[1])]

SyntaxError: invalid syntax (<ipython-input-104-d3a41e7781af>, line 1)

In [121]:
groups = [[list(zip(*logs)) for logs in [l for l in list(g)]] for k, g in groupby(zip(data, blk), lambda x: x[1])]

In [215]:
%%time
groups = [(k, [log_parser(l) for l in list(zip(*list(g)))[0]])
          for k, g in groupby(zip(data, blk), lambda x: x[1])]

Wall time: 46.6 s


In [379]:
%%time #######################
groups = [[k, list(list(zip(*list(g)))[0])] for k, g in 
          groupby(zip(data, blk), lambda x: x[1])]

Wall time: 13.8 s


In [235]:
from itertools import chain
from math import ceil
gg = [groups[0][1][i*3:i*3+3] for i in range(ceil(len(groups[0][1])/3))]

In [290]:
sublists = [(subgroup[0], [subgroup[1][i : i + 3] for i in range(len(subgroup[1])-2)])
             for subgroup in groups if len(subgroup[1]) >= 3]

In [399]:
%%time
three_len_logs = []

for subgroup in groups:
    key, logs = subgroup
    
    if len(logs) == 0:
        raise ValueError('Zero-length sequence exists')
    if len(logs) == 1:
        # 로그가 한 개인 경우 AE처럼 자기 자신을 예측하도록 설정
        logs += [logs[0], logs[0]]
    elif len(logs) == 2:
        # 로그가 두 개인 경우 뒤 로그를 한 개 더 복사
        logs += [logs[1]]
        
    for i in range(len(logs)-2):
        three_len_logs.append((key, logs[i:i+3]))

Wall time: 11.4 s


In [417]:
three_len_logs[27]

('blk_-9073992586687739851',
 ['081109 203524 146 INFO dfs.DataNode$PacketResponder: PacketResponder 0 for block blk_-9073992586687739851 terminating',
  '081109 203524 146 INFO dfs.DataNode$PacketResponder: Received block blk_-9073992586687739851 of size 11977 from /10.250.14.196',
  '081109 203524 147 INFO dfs.DataNode$PacketResponder: PacketResponder 1 for block blk_-9073992586687739851 terminating'])

In [456]:
# check length
lens = [log_parser(data[i]) for i in range(len(data))]

In [457]:
lens = [len(l.split()) for l in lens]

In [458]:
np.mean(lens)

15.600405994746815

In [459]:
np.max(lens)

31

In [425]:
import vocab
import os
vocab_path = './data/vocab.txt'
def load_vocab():
    if not os.path.exists(vocab_path):
        print("Creating vocab.txt...")
        
        print('done.')
    with open(vocab_path, 'r') as f:
        vocabs = f.read().splitlines()
    return vocabs

In [426]:
vocabs = load_vocab()

In [430]:
[[vocabs.index(l) for l in log_parser(log).split()] for log in three_len_logs[27][1]]

[[3, 3, 3, 4, 5, 6, 23, 23, 3, 24, 9, 10, 25],
 [3, 3, 3, 4, 5, 6, 23, 26, 9, 10, 27, 28, 3, 29, 12],
 [3, 3, 3, 4, 5, 6, 23, 23, 3, 24, 9, 10, 25]]

In [437]:
lll = [len(log_parser(d)) for d in data]

In [432]:
def abc(a, b, c):
    return a+b+c

In [434]:
abc((1, 2, 3))

TypeError: abc() missing 2 required positional arguments: 'b' and 'c'

In [394]:
tt1 = (1, 2, 3, 4)
tt2 = (2,3,4,5)

In [398]:
test+=tt1

test

[(1, 2, 3, 4), (1, 2, 3, 4), 1, 2, 3, 4]

In [ ]:
tt

In [388]:
three_len_logs[1]

('blk_-9073992586687739851',
 [['081109 203523 143 INFO dfs.DataNode$DataXceiver: Receiving block blk_-9073992586687739851 src: /10.250.19.102:37673 dest: /10.250.19.102:50010',
   '081109 203523 143 INFO dfs.DataNode$DataXceiver: Receiving block blk_-9073992586687739851 src: /10.250.19.102:37673 dest: /10.250.19.102:50010',
   '081109 203523 143 INFO dfs.DataNode$DataXceiver: Receiving block blk_-9073992586687739851 src: /10.250.19.102:37673 dest: /10.250.19.102:50010']])

In [380]:
groups[0]

['blk_-1608999687919862906',
 ['081109 203518 143 INFO dfs.DataNode$DataXceiver: Receiving block blk_-1608999687919862906 src: /10.250.19.102:54106 dest: /10.250.19.102:50010',
  '081109 203518 35 INFO dfs.FSNamesystem: BLOCK* NameSystem.allocateBlock: /mnt/hadoop/mapred/system/job_200811092030_0001/job.jar. blk_-1608999687919862906',
  '081109 203519 143 INFO dfs.DataNode$DataXceiver: Receiving block blk_-1608999687919862906 src: /10.250.10.6:40524 dest: /10.250.10.6:50010',
  '081109 203519 145 INFO dfs.DataNode$DataXceiver: Receiving block blk_-1608999687919862906 src: /10.250.14.224:42420 dest: /10.250.14.224:50010',
  '081109 203519 145 INFO dfs.DataNode$PacketResponder: PacketResponder 1 for block blk_-1608999687919862906 terminating',
  '081109 203519 145 INFO dfs.DataNode$PacketResponder: PacketResponder 2 for block blk_-1608999687919862906 terminating',
  '081109 203519 145 INFO dfs.DataNode$PacketResponder: Received block blk_-1608999687919862906 of size 91178 from /10.250.10

In [320]:
sum([1 if len(g[1])>=3 else 0 for g in groups])

421964

In [314]:
len(blk)

8712428

In [316]:
len(groups)

6541067

In [ ]:
for i in range(30, 100):
    print('---------------------------------------------------')
    print(groups[i][1])

In [319]:
groups[0][0]

'blk_-1608999687919862906'

In [264]:
sublists[2]

[('081109 203523 149 INFO dfs.DataNode$DataXceiver: 10.251.111.209:50010 Served block blk_-1608999687919862906 to /10.250.19.102',)]

In [229]:
len(groups[0][1])

22

In [214]:
groups[-2]

('blk_5271023434684512497',
 ['NUM NUM NUM info dfs datanode packetresponder received block BLK of size NUM from IP'])

In [168]:
groups['blk_-9073992586687739851']

TypeError: list indices must be integers or slices, not str

In [ ]:
list(zip(*groups[0]))

In [ ]:
[ for x in y for y in x]

In [95]:
groups[0]

['NUM NUM NUM info dfs datanode dataxceiver receiving block BLK src IP dest IP',
 'NUM NUM NUM info dfs fsnamesystem block namesystem allocateblock mnt hadoop mapred system job NUM NUM job jar BLK',
 'NUM NUM NUM info dfs datanode dataxceiver receiving block BLK src IP dest IP',
 'NUM NUM NUM info dfs datanode dataxceiver receiving block BLK src IP dest IP',
 'NUM NUM NUM info dfs datanode packetresponder packetresponder NUM for block BLK terminating',
 'NUM NUM NUM info dfs datanode packetresponder packetresponder NUM for block BLK terminating',
 'NUM NUM NUM info dfs datanode packetresponder received block BLK of size NUM from IP',
 'NUM NUM NUM info dfs datanode packetresponder received block BLK of size NUM from IP',
 'NUM NUM NUM info dfs datanode packetresponder packetresponder NUM for block BLK terminating',
 'NUM NUM NUM info dfs datanode packetresponder received block BLK of size NUM from IP',
 'NUM NUM NUM info dfs fsnamesystem block namesystem addstoredblock blockmap updated

In [22]:
from collections import Counter

In [23]:
ctr = Counter()

In [24]:
ctr.update(['<pad>', '<EOS>', 'num', 'id', 'ip'])

In [25]:
ctr

Counter({'<pad>': 1, '<EOS>': 1, 'num': 1, 'id': 1, 'ip': 1})